In [1]:
import os
import chromadb
import pypdf
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer


d:\Python3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 初始化 ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="pdf_documents")

# 選擇適合中文的語意檢索模型
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [3]:
def extract_text_from_pdf(pdf_path):
    """ 從 PDF 提取文字內容 """
    text = ""
    with open(pdf_path, "rb") as file:
        reader = pypdf.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

In [4]:
def split_text(text, chunk_size=500, chunk_overlap=50):
    """ 切分文本為小段落 """
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_text(text)

In [5]:
def text_to_vector(text):
    """ 將文本轉換為向量 """
    return model.encode(text).tolist()

In [6]:
def store_vectors_in_chroma(text_chunks, file_name):
    """ 儲存文本向量到 ChromaDB，並包含文件名稱 """
    for idx, chunk in enumerate(text_chunks):
        collection.add(
            ids=[f"{file_name}_{idx}"],
            embeddings=[text_to_vector(chunk)],
            metadatas=[{"text": chunk, "file_name": file_name}]
        )

In [ ]:
def process_pdfs(pdf_folder):
    """ 處理指定資料夾內的所有 PDF，並存入向量資料庫 """
    chroma_client.clear_system_cache()
    chroma_client.reset()
    for file in os.listdir(pdf_folder):
        if file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, file)
            print(f"處理文件: {file}")
            pdf_text = extract_text_from_pdf(pdf_path)
            text_chunks = split_text(pdf_text)
            store_vectors_in_chroma(text_chunks, file)

In [8]:
def search_chroma(query, top_k=3):
    """ 使用 ChromaDB 查詢相似文本，並回傳對應的 PDF 文件名稱 """
    query_vector = text_to_vector(query)
    results = collection.query(query_embeddings=[query_vector], n_results=top_k)
    
    matched_texts = [r["text"] for r in results["metadatas"][0]]
    file_names = [r["file_name"] for r in results["metadatas"][0]]
    
    return list(zip(file_names, matched_texts))

In [ ]:
# 🔹 處理多個 PDF 文件
process_pdfs("D:\.vscode\專題\product infomation")  # 你的 PDF 資料夾路徑

處理文件: DE-291-1 DE-293 工作桌.pdf
處理文件: SADES DIABLO 暗黑鬥狼RGB REALTEK 電競耳麥 7.1 (USB) SA-916.pdf
處理文件: W202 人體工學椅.pdf
處理文件: [折疊收納]懶人折疊桌.pdf
處理文件: 世界上最透明的故事（日本出版界話題作，只有紙本書可以體驗的感動）.pdf
處理文件: 羅技 Logitech H340 USB耳機麥克風.pdf


In [ ]:
# 🔹 測試查詢
query = ""
search_results = search_chroma(query, 5)
with open('a.txt', 'w', encoding='utf-8') as f:
    for i, (file_name, result) in enumerate(search_results):
        f.write(f"來自文件: {file_name}\n相關段落 {i+1}: {result}\n\n")